In [1]:
# Import necessary libraries
import pandas as pd
import requests

# Load the validation-formatted Excel file
excel_path = r'val_formated.xlsx'  # Change this if your file is elsewhere
df_val_formated = pd.read_excel(excel_path)

# Display the first few rows to verify
print(df_val_formated.head())

                                                text  \
0                                           Em Lajes   
1  Corte e remoção de gradeamento existente, em c...   
2                                Linteis de Fundação   
3  Betão C25/30 em Lajes de Transição, incluindo ...   
4  Fornecimento e aplicação de sistema de imperme...   

                                              labels  
0  (['ESTRUTURA BETÃO ARMADO'], ['7.8 Lajes'], ['...  
1  (['DEMOLIÇÕES E CONTENÇÃO DE FACHADA'], ['2.3 ...  
2  (['FUNDAÇÕES'], ['5.1 Betão de Limpeza'], ['Th...  
3  (['ESTRUTURA BETÃO ARMADO'], ['7.8 Lajes'], ['...  
4  (['DIVERSOS'], ['Impermeabilização - Telas Ben...  


In [2]:
# Define the static instruction and context
instruction = (
    "Segue-se uma instrução que descreve uma tarefa, emparelhada com uma entrada que fornece mais contexto. "
    "Escreva uma resposta que complete adequadamente o pedido\n\n"
    "### Instruction:\n"
    "Classifica o input do user em Capítulo, Subcapítulo e Item.\n\n"
    "### Input:\n"
)

# Create the formatted prompt
df_val_formated['prompt'] = (
    instruction +
    df_val_formated['text'].astype(str) +  # Changed from 'input' to 'text'
    "\n\n### Response:\n" +
    df_val_formated['labels'].astype(str)   # Changed from 'response' to 'labels'
)

In [3]:
print(df_val_formated.head())

                                                text  \
0                                           Em Lajes   
1  Corte e remoção de gradeamento existente, em c...   
2                                Linteis de Fundação   
3  Betão C25/30 em Lajes de Transição, incluindo ...   
4  Fornecimento e aplicação de sistema de imperme...   

                                              labels  \
0  (['ESTRUTURA BETÃO ARMADO'], ['7.8 Lajes'], ['...   
1  (['DEMOLIÇÕES E CONTENÇÃO DE FACHADA'], ['2.3 ...   
2  (['FUNDAÇÕES'], ['5.1 Betão de Limpeza'], ['Th...   
3  (['ESTRUTURA BETÃO ARMADO'], ['7.8 Lajes'], ['...   
4  (['DIVERSOS'], ['Impermeabilização - Telas Ben...   

                                              prompt  
0  Segue-se uma instrução que descreve uma tarefa...  
1  Segue-se uma instrução que descreve uma tarefa...  
2  Segue-se uma instrução que descreve uma tarefa...  
3  Segue-se uma instrução que descreve uma tarefa...  
4  Segue-se uma instrução que descreve uma tarefa..

In [4]:
import requests
import pandas as pd

# Load your dataset
df = pd.read_excel('val_formated.xlsx')

# Define the API endpoint
API_URL = "http://127.0.0.1:1234/v1/chat/completions"

# Function to get predictions
def get_prediction(text):
    headers = {
        "Content-Type": "application/json"
    }
    
    payload = {
        "model": "unsloth",
        "messages": [
            {
                "role": "system",
                "content": "Classifica o input do user em Capítulo, Subcapítulo e Item."
            },
            {
                "role": "user",
                "content": f"### Input:\n{text}\n\n### Response:"
            }
        ],
        "temperature": 0.01,
        "max_tokens": 100,
        "top_p": 0.95
    }
    
    try:
        response = requests.post(API_URL, headers=headers, json=payload)
        response.raise_for_status()
        return response.json()['choices'][0]['message']['content']
    except Exception as e:
        print(f"Error: {str(e)}")
        return None


In [5]:
# Test on first 5 data points
test_df = df_val_formated
test_df['prediction'] = test_df['text'].apply(get_prediction)

# Display results
print("Original Text and Predictions:")
print(test_df[['text', 'prediction']])

Original Text and Predictions:
                                                   text  \
0                                              Em Lajes   
1     Corte e remoção de gradeamento existente, em c...   
2                                   Linteis de Fundação   
3     Betão C25/30 em Lajes de Transição, incluindo ...   
4     Fornecimento e aplicação de sistema de imperme...   
...                                                 ...   
1794                                     Trelica Tipo 2   
1795                       Betão C25/30 - XC1(P) 22 S3    
1796  Remoção de materiais inertes  existente na zon...   
1797  Em perfis tubulares para contraventamento do a...   
1798  Laje térrea em betão armado, para bancada do a...   

                                             prediction  
0     Chapter: ESTRUTURA BETÃO ARMADO | Subchapter: ...  
1     Chapter: DEMOLIÇÕES E CONTENÇÃO DE FACHADA | S...  
2     Chapter: FUNDAÇÕES | Subchapter: 5.2 Fundações...  
3     Chapter: ESTRUTURA BET

In [11]:
print(test_df.columns)

Index(['text', 'labels', 'prompt', 'prediction'], dtype='object')


In [15]:
print(test_df.head())

                                                                                                                                                                                                                                                                                                                                          text  \
0                                                                                                                                                                                                                                                                                                                                     Em Lajes   
1                                                                                                                                                                        Corte e remoção de gradeamento existente, em cantoneira de ferro metalizado, com o acondicionamento em local a designar pela Fiscalização para posterior re

In [16]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import precision_score, recall_score, f1_score, hamming_loss
import ast

# Already parsed `labels` into tuples of lists: (chapter, subchapter, item)
test_df['labels'] = test_df['labels'].apply(ast.literal_eval)

# Extract all levels as full lists
test_df['true_chapter'] = test_df['labels'].apply(lambda x: x[0])
test_df['true_subchapter'] = test_df['labels'].apply(lambda x: x[1])
test_df['true_item'] = test_df['labels'].apply(lambda x: x[2])

# Parse prediction string into lists (wrap single values into lists for consistency)
def parse_prediction(pred):
    try:
        parts = pred.split(" | ")
        chapter = [parts[0].replace("Chapter:", "").strip()]
        subchapter = [parts[1].replace("Subchapter:", "").strip()]
        item = [i.strip() for i in parts[2].replace("Item:", "").split(',')]
        return chapter, subchapter, item
    except:
        return [], [], []

# Extract predicted levels
test_df[['pred_chapter', 'pred_subchapter', 'pred_item']] = test_df['prediction'].apply(
    lambda x: pd.Series(parse_prediction(x))
)


In [17]:
pd.set_option('display.max_colwidth', None)  # Show full column width
print(test_df[['true_chapter', 'true_subchapter', 'true_item']].head())


                          true_chapter  \
0             [ESTRUTURA BETÃO ARMADO]   
1  [DEMOLIÇÕES E CONTENÇÃO DE FACHADA]   
2                          [FUNDAÇÕES]   
3             [ESTRUTURA BETÃO ARMADO]   
4                           [DIVERSOS]   

                                                                                                  true_subchapter  \
0                                                                                                     [7.8 Lajes]   
1  [2.3 Demolição do Edifício existente incluindo acumulação e transporte dos produtos para vazadouro autorizado]   
2                                                                                          [5.1 Betão de Limpeza]   
3                                                                                                     [7.8 Lajes]   
4                                                                 [Impermeabilização - Telas Bentoníticas Voltex]   

                                            

In [14]:
print(test_df[['pred_chapter', 'pred_subchapter', 'pred_item']].head())


                          pred_chapter                     pred_subchapter  \
0             [ESTRUTURA BETÃO ARMADO]                         [7.8 Lajes]   
1  [DEMOLIÇÕES E CONTENÇÃO DE FACHADA]       [2.4 Demolições de Elementos]   
2                          [FUNDAÇÕES]             [5.2 Fundações Diretas]   
3             [ESTRUTURA BETÃO ARMADO]                         [7.8 Lajes]   
4                           [DIVERSOS]  [Impermeabilização - Telas em PVC]   

                                                                pred_item  
0  [Laje maciças, Laje fungiformes Maciças, Laje fungiformes Aligeiradas]  
1                                                  [Caixilharia Exterior]  
2                                                     [Vigas de Fundação]  
3                                                          [Laje maciças]  
4                                                      [Throwaway - Item]  


In [18]:
def multilabel_metrics(y_true_list, y_pred_list, level_name=""):
    mlb = MultiLabelBinarizer()
    y_true_bin = mlb.fit_transform(y_true_list)
    y_pred_bin = mlb.transform(y_pred_list)  # Only keeps known labels in test set

    precision = precision_score(y_true_bin, y_pred_bin, average='micro', zero_division=0)
    recall = recall_score(y_true_bin, y_pred_bin, average='micro', zero_division=0)
    f1 = f1_score(y_true_bin, y_pred_bin, average='weighted', zero_division=0)
    hamming = hamming_loss(y_true_bin, y_pred_bin)

    return {
        f"{level_name}_precision_micro": precision,
        f"{level_name}_recall_micro": recall,
        f"{level_name}_f1_weighted": f1,
        f"{level_name}_hamming_loss": hamming
    }

# Apply for each level
chapter_metrics = multilabel_metrics(test_df['true_chapter'], test_df['pred_chapter'], "chapter")
subchapter_metrics = multilabel_metrics(test_df['true_subchapter'], test_df['pred_subchapter'], "subchapter")
item_metrics = multilabel_metrics(test_df['true_item'], test_df['pred_item'], "item")

# Combine all levels for overall evaluation
overall_true = test_df['true_chapter'] + test_df['true_subchapter'] + test_df['true_item']
overall_pred = test_df['pred_chapter'] + test_df['pred_subchapter'] + test_df['pred_item']
overall_true_grouped = [ch + sub + it for ch, sub, it in zip(test_df['true_chapter'], test_df['true_subchapter'], test_df['true_item'])]
overall_pred_grouped = [ch + sub + it for ch, sub, it in zip(test_df['pred_chapter'], test_df['pred_subchapter'], test_df['pred_item'])]

overall_metrics = multilabel_metrics(overall_true_grouped, overall_pred_grouped, "overall")


# Merge results
all_metrics = {**chapter_metrics, **subchapter_metrics, **item_metrics, **overall_metrics}

import pprint
pprint.pprint(all_metrics)


{'chapter_f1_weighted': 0.8857390323837246,
 'chapter_hamming_loss': 0.018945710579951824,
 'chapter_precision_micro': 0.9314447592067989,
 'chapter_recall_micro': 0.8509316770186336,
 'item_f1_weighted': 0.644881707094862,
 'item_hamming_loss': 0.011774929734046301,
 'item_precision_micro': 0.8223620522749274,
 'item_recall_micro': 0.599083215796897,
 'overall_f1_weighted': 0.6730738640011897,
 'overall_hamming_loss': 0.014420481402537782,
 'overall_precision_micro': 0.8665527714502658,
 'overall_recall_micro': 0.6112747723620782,
 'subchapter_f1_weighted': 0.5508398736539646,
 'subchapter_hamming_loss': 0.0167851187167474,
 'subchapter_precision_micro': 0.8515679442508711,
 'subchapter_recall_micro': 0.4525925925925926}


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:875: UserWarning: unknown class(es) ['###  TRABALHOS PREPARATÓRIOS E MONTAGEM DE ESTALEIRO', '- Fornecimento e montagem de cofragens metálicas para execução de núcleos de betão armado, incluindo escoramentos, descofragens e limpeza dos elementos, conforme especificações técnicas do projeto.\n\n###  ESTRUTURA BETÃO ARMADO', 'CONTENÇÃO DE FACHADA', 'CONTENÇÕES, FUNDAÇÕES', 'Elemento de fundação horizontal (laje de fundação) para implantação da estrutura do edifício, incluindo todos os trabalhos necessários à sua boa execução.\n\n###  FUNDAÇÕES', 'None', 'TRABALHOS PREPARATÓRIOS E COMPLEMENTARES'] will be ignored
  warnings.warn(
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:875: UserWarning: unknown class(es) ['1.1 Estaleiro, 1.2 Equipamento Básico da Empreitada', '1.1 Estaleiro, 1.2 Equipamentos, 1.3 Serviços', '1.1 Estaleiro, 1.2 Plano de Segurança e Saúde', '1.1 Estaleiro, 1.2 Plano

In [19]:
rounded_metrics = {k: round(v, 3) for k, v in all_metrics.items()}

import pprint
pprint.pprint(rounded_metrics)


{'chapter_f1_weighted': 0.886,
 'chapter_hamming_loss': 0.019,
 'chapter_precision_micro': 0.931,
 'chapter_recall_micro': 0.851,
 'item_f1_weighted': 0.645,
 'item_hamming_loss': 0.012,
 'item_precision_micro': 0.822,
 'item_recall_micro': 0.599,
 'overall_f1_weighted': 0.673,
 'overall_hamming_loss': 0.014,
 'overall_precision_micro': 0.867,
 'overall_recall_micro': 0.611,
 'subchapter_f1_weighted': 0.551,
 'subchapter_hamming_loss': 0.017,
 'subchapter_precision_micro': 0.852,
 'subchapter_recall_micro': 0.453}


In [40]:
comparison_df = test_df[[
    'true_chapter', 'pred_chapter',
    'true_subchapter', 'pred_subchapter',
    'true_item', 'pred_item'
]]

# Optional: display first few rows
print(comparison_df.head(10))

                        true_chapter                       pred_chapter  \
0             ESTRUTURA BETÃO ARMADO             ESTRUTURA BETÃO ARMADO   
1  DEMOLIÇÕES E CONTENÇÃO DE FACHADA  DEMOLIÇÕES E CONTENÇÃO DE FACHADA   
2                          FUNDAÇÕES                          FUNDAÇÕES   
3             ESTRUTURA BETÃO ARMADO                          FUNDAÇÕES   
4                           DIVERSOS                           DIVERSOS   
5             ESTRUTURA BETÃO ARMADO             ESTRUTURA BETÃO ARMADO   
6             ESTRUTURA BETÃO ARMADO             ESTRUTURA BETÃO ARMADO   
7                               None                          FUNDAÇÕES   
8             ESTRUTURA BETÃO ARMADO             ESTRUTURA BETÃO ARMADO   
9                         CONTENÇÕES             ESTRUTURA BETÃO ARMADO   

                                     true_subchapter  \
0                                          7.8 Lajes   
1  2.3 Demolição do Edifício existente incluindo ...   
2     